In [1]:
import pyscf
import pyscf.tools
import pyscf.lo

from orbitalpartitioning import *

In [3]:
molecule = """
Fe  5.48 1.15 -8.03
S   4.05 -0.61 -8.75
S   7.49 0.42 -9.04
Fe  6.04 -1.22 -9.63
S   5.47 1.25 -5.58
S   4.63 3.28 -8.77
S   5.75 -1.50 -12.05
S   6.86 -3.41 -8.86
C   5.51 4.45 -7.51
H   6.49 4.83 -7.92
H   4.87 5.33 -7.25
H   5.72 3.84 -6.59
C   3.60 1.70 -5.54
H   3.01 0.80 -5.82
H   3.28 2.06 -4.52
H   3.42 2.48 -6.31
C   5.21 -4.22 -9.46
H   5.10 -4.01 -10.55
H   5.21 -5.32 -9.26
H   4.37 -3.72 -8.93
C   7.63 -1.85 -12.24
H   7.90 -2.06 -13.31
H   8.20 -0.96 -11.86
H   7.89 -2.72 -11.59
"""

basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
mf = pyscf.scf.UHF(pymol).newton()
# mf = pyscf.scf.RHF(pymol).newton()

# mf.damp = 0.5
# mf.diis_start_cycle = 8

mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
# P = np.load("UHF_DM.npy")
# mf.init_guess = "sad"
# mf.kernel(P)
mf.kernel()
np.save("UHF_DM", mf.make_rdm1())
print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)



symmetry:  C1


******** <class 'pyscf.scf.uhf.UHF'> Newton solver flags ********
SCF tol = 1e-08
conv_tol_grad = 1e-05
max. SCF cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_cycle_inner = 12
max_stepsize = 0.05
ah_start_tol = 1e+09
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_start_cycle = 1
ah_max_cycle = 40
ah_grad_trust_region = 2.5
kf_interval = 4
kf_trust_region = 5
canonicalization = True
max_memory 4000 MB (current use 0 MB)
  alpha nocc = 98  HOMO = 0.199929094185571  LUMO = 0.205421971066349
  beta  nocc = 88  HOMO = -0.262446756477781  LUMO = 0.0599938177317816

WARN: system HOMO 0.199929094185571 >= system LUMO 0.0599938177317816

Initial guess E= -5039.93568418245  |g|= 5.2673
macro= 0  E= -5063.15092894791  delta_E= -23.2152  |g|= 3.14642  3 KF 15 JK
macro= 1  E= -5066.25755253506  delta_E= -3.10662  |g|= 0.758744  3 KF 15 JK
macro= 2  E= -5067.2980897525  delta_E= -1.04054  |g|= 0.342875  3 KF 17 JK
macro= 3

In [4]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
frag4 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("3p"):
            frag3.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 3:
        if ao[2] in ("3d"):
            frag4.append(ao_idx)
            full.append(ao_idx)



frags = [frag1, frag2, frag3, frag4]
print(frags)

[[14, 15, 16, 17, 18], [38, 39, 40], [56, 57, 58], [81, 82, 83, 84, 85]]


# Build DMET Active Space

In [6]:
S = mf.get_ovlp()
Co = mf.mo_coeff[0][:,mf.mo_occ[0]==1]
Cv = mf.mo_coeff[0][:,mf.mo_occ[0]==0]

orbitals, init_fspace, clusters = orbitalpartitioning.dmet_clustering(Co, Cv, frags, S)


# F = mf.get_fock()
# orbitals = orbitalpartitioning.canonicalize(orbitals, F)


Cenv = orbitals[0]
Cact = np.hstack((orbitals[1:1+len(frags)]))
pyscf.tools.molden.from_mo(mf.mol, "scr/Cact.molden", Cact)

clusters = [[i+1 for i in j] for j in clusters]

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


 Create DMET active space by projecting  286 MOs onto   16 fragment orbitals
 Dmet active space has the following dimensions:
   Environment (occupied)   :    82
   Active (occupied)        :    16
   Active (virtual)         :    16
   Environment (virtual)    :   172
 Partition  286 orbitals into a total of    4 fragments
 Create DMET active space by projecting  286 MOs onto    5 fragment orbitals
 Dmet active space has the following dimensions:
   Environment (occupied)   :    93
   Active (occupied)        :     5
   Active (virtual)         :     5
   Environment (virtual)    :   183
 Create DMET active space by projecting  286 MOs onto    3 fragment orbitals
 Dmet active space has the following dimensions:
   Environment (occupied)   :    95
   Active (occupied)        :     3
   Active (virtual)         :     3
   Environment (virtual)    :   185
 Create DMET active space by projecting  286 MOs onto    3 fragment orbitals
 Dmet active space has the following dimensions:
   Envir

# Make Integrals

In [8]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)

(286, 82)
(286, 32)
(286, 286)


In [10]:
import numpy as np
Ccmf = np.load("Ccmf.npy")
pyscf.tools.molden.from_mo(mf.mol, "scr/Ccmf.molden", Ccmf)


FileNotFoundError: [Errno 2] No such file or directory: 'src/Ccmf.molden'